![title](imagens/logo_flisol_demo01_Teste.png)

# Objetivo da Demo

Faremos Scraping (raspagem) no site da Amazon Americana para buscar preços, configurações e avaliações de notebooks, fazendo todo tratamento necessário nos dados vindos do site.

## Carregando os Pacotes "Padrão" Necessários

In [44]:
import bs4
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## Web Scraping

O primeiro passo é você acessar o endereço abaixo:

https://www.amazon.com/s?k=laptops&page=

Esse endereço é o resultado da busca por "laptops". Inspecionaremos o código fonte da página.

Coletaremos as informações relevantes das primeiras 10 páginas do resultado da pesquisa, pois isso nos fornecerá dados suficientes para a análise. 
Criaremos então um dataframe a partir dos dados coletados. 

Criaremos funções mostrando como organizar melhor o código!

## Função Leitura da Página da Amanzon na Pesquisa
### Esta função define o cabeçalho da conexão ao site da Amazon e faz a requisição para extrair os dados

In [54]:
def faz_request(num_page):
    
    headers01 = {"User-Agent": "Mozilla/5.0 (Windows NT 11.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
       "Accept-Encoding": "gzip, deflate", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
       "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1","Accept-Language": "en-US,en;q=0.5"}


    # Cabeçalho para evitar problema de conexão não segura ao site da Amazon
    
    
    headers02 = {
                'User-Agent': ('Mozilla/5.0 (X11; Linux x86_64)'
                    'AppleWebKit/537.36 (KHTML, like Gecko)'
                    'Chrome/44.0.2403.157 Safari/537.36'),
                    'Accept-Language': 'en-US, en;q=0.5'
                }

    headers03 = {
                
                'dnt': '1',
                'upgrade-insecure-requests': '1',
                'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'sec-fetch-site': 'same-origin',
                'sec-fetch-mode': 'navigate',
                'sec-fetch-user': '?1',
                'sec-fetch-dest': 'document',
                'referer': 'https://www.amazon.com/',
                'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
                }


    # Request para cada página do resultado de pesquisa
    # No parâmetro num_page, será passado o número da página a ser lida
    requisicao = requests.get('https://www.amazon.com/s?k=laptops&page='+str(num_page), headers = headers03)
    
    return requisicao

## Função Extração do conteúdo da leitura da página numéro tal da pesquisa 

In [46]:
# Função para extrair o conteúdo
def extrai_conteudo(requisicao):
    return requisicao.content

## Função Leitura do HTML da página numéro tal da pesquisa da Pesquisa


In [47]:
# Função para o parse do conteúdo extraído do site da Amazon
def formata_conteudo(conteudo):
    return BeautifulSoup(conteudo, 'lxml')
    

## Função Extração do Nome do Notebook do HTML da Página de Pesquisa da Amanzon
#### Procuramos normalmente pela classe do CSS da Tag

In [48]:
# Função para extrair o nome do notebook
def extrai_nome(div):
    
    # Busca pela tag span, classe 'a-size-medium a-color-base a-text-normal', onde está o nome do notebook                                                      
    span_nome = div.find('span', attrs = {'class':'a-size-medium a-color-base a-text-normal'})
       
    # Se o valor não for vazio, extrai o valor (nome)
    if span_nome is not None:
        return span_nome.text
    else:
        return 'no-info' 

## Função Extração do Preço do Notebook do HTML da Página de Pesquisa da Amanzon
#### Procuramos normalmente pela classe do CSS da Tag

In [49]:
# Função para extrair o preço
def extrai_preco(div):
    
    # Busca pela tag span, classe 'a-offscreen', onde está o preço do notebook
    span_preco = div.find('span', attrs = {'class':'a-offscreen'})
    
    # Se o valor não for vazio, extrai o valor (preço)
    if span_preco is not None:
        return span_preco.text
    else:
        return 'no-info'

## Função Extração da Avaliação do Notebook do HTML da Página de Pesquisa da Amanzon
#### Procuramos normalmente pela classe do CSS da Tag

In [50]:
# Função para extrair a avaliação dos clientes
def extrai_aval(div):
    
    # Busca pela tag span, classe 'a-icon-alt', onde está a avaliação do notebook
    span_aval = div.find('span', attrs = {'class':'a-icon-alt'})
    
    # Se o valor não for vazio, extrai o valor (avaliação)
    if span_aval is not None:
        return span_aval.text
    else:
        return 'no-info'  

In [51]:
# Lista para receber os dados
# dados_notebooks = []

# Cria o dataframe para receber os dados
df_dados_notebooks = pd.DataFrame(columns = ["Nome", "Preco", "Avaliacao"])

In [52]:
# nome = 'Nome'
# preco = "1"
# aval = "4"
# #dados_notebooks = dados_notebooks.append([nome, preco, aval])


# novo_notebook = pd.Series({"Nome":'Fulano', 
#                            "Preco":'1', 
#                            "Avaliacao":'4'})       
# dados_notebooks = pd.concat([df_dados_notebooks,novo_notebook.to_frame().T],ignore_index=True)



## Faz o Loop para a Leitura das 10 Primeiras Páginas de Pesquisa da Amazon

In [72]:
# Loop por cada página retornada no scraping...
print("\nExecutando Web Scraping...\n")

# Loop
for num_page in range(1,10):
    
    # Faz a request
    requisicao = faz_request(num_page)
    
    # Exrai o conteúdo
    conteudo = extrai_conteudo(requisicao)
    
    # Formata o conteúdo
    soup = formata_conteudo(conteudo)   

    #print(soup)
    
    # Loop pelo conteúdo  
    # s-main-slot s-result-list s-search-results sg-row
    # a-section a-spacing-small a-spacing-top-small
    #for info in soup.findAll('div', attrs = {'class':'sg-col-inner'}):
    for info in soup.findAll('div', attrs = {'class':'a-section a-spacing-small a-spacing-top-small'}):
        
        # Extrai o nome do notebook
        nome = extrai_nome(info)
        
        # Extrai o preço do notebook
        preco = extrai_preco(info)
        
        # Extrai a avaliação do notebook
        aval = extrai_aval(info)
        
        novo_notebook = pd.Series({"Nome":nome, 
                           "Preco":preco, 
                           "Avaliacao":aval})       
        # dados_notebooks = pd.concat([dados_notebooks,novo_notebook.to_frame().T],ignore_index=True)
        df_dados_notebooks = pd.concat([df_dados_notebooks,novo_notebook.to_frame().T],ignore_index=True)

        # Adiciona os dados à lista
        # dados_notebooks = dados_notebooks.append([nome, preco, aval])

    print("\nExtração da Página " + str(num_page) + " Concluída.\n")

print("\nExtração Concluída.\n")


Executando Web Scraping...


Extração da Página 1 Concluída.


Extração da Página 2 Concluída.


Extração da Página 3 Concluída.


Extração da Página 4 Concluída.


Extração da Página 5 Concluída.


Extração da Página 6 Concluída.


Extração da Página 7 Concluída.


Extração da Página 8 Concluída.


Extração da Página 9 Concluída.


Extração Concluída.



Mostra as Dimensões do DataFrame (Nº de Linhas e Colunas)

In [73]:
# Shape
df_dados_notebooks.shape

(257, 3)

Visualiza as 5 Primeiras Linhas do DataFrame

In [74]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco,Avaliacao
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 1...",$499.99,4.3 out of 5 stars
4,no-info,no-info,no-info


In [75]:
df_dados_notebooks.head(20)

,Nome,Preco,Avaliacao
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 1...",$499.99,4.3 out of 5 stars
4,no-info,no-info,no-info
5,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars
6,"Lenovo 15.6"" IdeaPad 1 Laptop, AMD Dual-core P...",$399.00,4.2 out of 5 stars
7,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",$383.00,4.4 out of 5 stars
8,"HP 2021 Stream 14"" HD SVA Laptop Computer, Int...",$289.00,4.2 out of 5 stars
9,"Lenovo Newest IdeaPad 1i 14"" HD Laptop, 4GB RA...",$228.95,4.4 out of 5 stars


Mostra as Tipos de Dados do DataFrame

In [76]:
# Visualizando os tipos de dados
df_dados_notebooks.dtypes

Nome         object
Preco        object
Avaliacao    object
dtype: object

Perfeito. Agora precisamos limpar os dados. Vamos à faxina.

## Limpeza da Coluna de Preços

Começaremos a limpeza pela coluna de preços.

Mostra Total de Registros Por Preço

In [77]:
df_dados_notebooks['Preco'].value_counts(dropna = False)

no-info    21
$399.00    17
$399.98    15
$499.99    13
$185.99    10
           ..
$639.00     1
$229.00     1
$289.95     1
$439.99     1
$729.00     1
Name: Preco, Length: 113, dtype: int64

Percebemos que temos Registros sem preço!

In [78]:
df_dados_notebooks[df_dados_notebooks['Preco'] == 'no-info'].count()[0]

21

Como temos vários registros com preço = no_info, isto é, sem preço, vamos remover estes!

In [79]:
df_dados_notebooks = df_dados_notebooks[df_dados_notebooks['Preco'] != 'no-info']

Agora não devemos ter mais registros sem preço!

In [80]:
df_dados_notebooks['Preco'].value_counts(dropna = False)

$399.00    17
$399.98    15
$499.99    13
$185.99    10
$599.00     8
           ..
$639.00     1
$229.00     1
$289.95     1
$439.99     1
$729.00     1
Name: Preco, Length: 112, dtype: int64

## Função Limpeza do Valor do Preço
#### Existe símbolos ($) e Vírgulas. Necessário retirá-los e vamos mater os pontos decimais!

In [81]:
# Função para limpeza da coluna de preço
def limpa_preco(price):
    
    # Remove o '$'
    price = price.replace('$', '')  
    
    # Remove a ',' 
    price = price.replace(',', '') 
    
    # Converte o tipo de dado para numérico
    price = pd.to_numeric(price)   
    
    # Arredonda os valores
    price = price.round()   
    
    return price

## Vamos aplicar a Função limpa_preco utilizando a Instrução Apply
### É aplicado a função criando uma nova coluna chamada Preco_Final com o Preço Limpo!

In [82]:
df_dados_notebooks['Preco_Final'] = df_dados_notebooks['Preco'].apply(limpa_preco)

In [83]:
df_dados_notebooks['Preco_Final']

0      400.0
1      400.0
2      400.0
3      500.0
5      400.0
       ...  
252    619.0
253    679.0
254    729.0
255    430.0
256    599.0
Name: Preco_Final, Length: 236, dtype: float64

## Vamos remover os registros com preços menores que 150, pois devem ser acessórios


In [84]:
df_dados_notebooks = df_dados_notebooks[df_dados_notebooks['Preco_Final'] >= 150]

Vamos mostrar Nº de linhas e colunas atuais

In [85]:
df_dados_notebooks.shape

(231, 4)

Vamos mostrar as 10 primeiras linhas

In [86]:
df_dados_notebooks.head(20)

,Nome,Preco,Avaliacao,Preco_Final
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars,400.0
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars,400.0
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars,400.0
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 1...",$499.99,4.3 out of 5 stars,500.0
5,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",$399.98,4.3 out of 5 stars,400.0
6,"Lenovo 15.6"" IdeaPad 1 Laptop, AMD Dual-core P...",$399.00,4.2 out of 5 stars,399.0
7,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",$383.00,4.4 out of 5 stars,383.0
8,"HP 2021 Stream 14"" HD SVA Laptop Computer, Int...",$289.00,4.2 out of 5 stars,289.0
9,"Lenovo Newest IdeaPad 1i 14"" HD Laptop, 4GB RA...",$228.95,4.4 out of 5 stars,229.0
10,"Lenovo ThinkBook 15 Premium Business Laptop, A...",$648.99,4.7 out of 5 stars,649.0


Não precisamos mais da coluna antiga. Vamos removê-la

In [87]:
df_dados_notebooks.drop('Preco', axis = 1, inplace = True)

Vamos mostrar as 3 primeiras linhas

In [88]:
# Visualiza os dados
df_dados_notebooks.head(3)

,Nome,Avaliacao,Preco_Final
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0


## Limpeza da Coluna de Avaliações

A coluna de avaliações possui valores como **3.8 out of 5 stars**, então vamos retirar o final depois da pontuação em sim.

In [89]:
# Número de elementos por tipo de avaliação
df_dados_notebooks['Avaliacao'].value_counts(dropna = False)

4.3 out of 5 stars    60
4.2 out of 5 stars    49
4.4 out of 5 stars    37
4.6 out of 5 stars    19
5.0 out of 5 stars    13
4.0 out of 5 stars    12
4.5 out of 5 stars    11
4.1 out of 5 stars     7
4.7 out of 5 stars     7
3.7 out of 5 stars     4
no-info                3
3.8 out of 5 stars     2
3.0 out of 5 stars     2
3.9 out of 5 stars     2
3.3 out of 5 stars     1
4.8 out of 5 stars     1
1.0 out of 5 stars     1
Name: Avaliacao, dtype: int64

## Função Limpeza da Avaliação
### Onde houver a texto **no-info**, colocaremos 0 como valor

In [90]:
# Função para limpar as avaliações e remover no-info
def limpa_aval(aval):
    
    # Divide a avaliação por espaços
    aval = aval.split()
    
    # Extraímos a primeira coluna (de índice zero)
    aval = aval[0]
    
    # Verificamos o valor 
    if aval == 'no-info':
        aval = 0
        
    # Convertemos para o tipo inteiro    
    aval = pd.to_numeric(aval) 
    
    return aval

## Vamos aplicar a Função limpa_aval utilizando a Instrução Apply
### É aplicado a função criando uma nova coluna chamada Avaliacao_Final

In [91]:
# Criamos uma nova coluna com o valor processado
df_dados_notebooks['Avaliacao_Final'] = df_dados_notebooks['Avaliacao'].apply(limpa_aval)

In [92]:
# Shape
df_dados_notebooks.shape

(231, 4)

In [93]:
# Visualiza os dados
df_dados_notebooks.head(10)

,Nome,Avaliacao,Preco_Final,Avaliacao_Final
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0,4.3
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0,4.3
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0,4.3
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 1...",4.3 out of 5 stars,500.0,4.3
5,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",4.3 out of 5 stars,400.0,4.3
6,"Lenovo 15.6"" IdeaPad 1 Laptop, AMD Dual-core P...",4.2 out of 5 stars,399.0,4.2
7,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",4.4 out of 5 stars,383.0,4.4
8,"HP 2021 Stream 14"" HD SVA Laptop Computer, Int...",4.2 out of 5 stars,289.0,4.2
9,"Lenovo Newest IdeaPad 1i 14"" HD Laptop, 4GB RA...",4.4 out of 5 stars,229.0,4.4
10,"Lenovo ThinkBook 15 Premium Business Laptop, A...",4.7 out of 5 stars,649.0,4.7


In [94]:
# Não precisamos mais da coluna antiga. Vamos removê-la
df_dados_notebooks.drop('Avaliacao', axis = 1, inplace = True)

In [95]:
# Visualiza os dados
df_dados_notebooks.head(3)

,Nome,Preco_Final,Avaliacao_Final
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",400.0,4.3
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",400.0,4.3
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch ...",400.0,4.3


## Limpeza da Coluna Nome

A coluna de nome consiste em muitas informações sobre o produto, além de detalhes técnicos, como nome da marca, tamanho da tela, memória e assim por diante. Portanto, para fazer uma análise eficaz, precisamos extrair essas informações. Nosso próximo passo consiste nesse processo. Esta é a parte mais complicada deste Lab. Para extração, precisamos usar expressões regulares.

In [96]:
# Vamos visualizar a coluna Nome
df_dados_notebooks['Nome']

0      Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch ...
1      Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch ...
2      Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch ...
3      ASUS VivoBook 15 F515 Thin and Light Laptop, 1...
5      Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch ...
                             ...                        
252    2022 HP 15.6 FHD Laptop Computer, 11th Gen Int...
253    HP Pavilion Business Laptop, 15.6" HD Touchscr...
254    Lenovo IdeaPad 3 14" FHD Laptop, 11th Gen Inte...
255    2022 Acer Aspire 5 Slim Laptop, 15.6 inch FHD ...
256    Lenovo New Ideapad 3 14inch FHD Portable Lapto...
Name: Nome, Length: 231, dtype: object

Esses 3 pontinhos ao final indicam que a string é muito longa e por isso foi truncada pelo pandas. Mas somos resolvedores de problemas, pesquisamos na documentação Python e encontramos como resolver isso.

In [97]:
# Parâmetro para imprimir nomes completos
pd.set_option('display.max_colwidth', None)

In [98]:
pd.set_option('display.max_rows', None)

In [99]:
# Vamos visualizar a coluna Nome
df_dados_notebooks['Nome']

0                                   Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories
1                                   Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories
2                                   Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories
3                        ASUS VivoBook 15 F515 Thin and Light Laptop, 15.6” FHD Display, Intel Core i3-1005G1 Processor, Fingerprint Reader, Backlit Keyboard, Windows 10 Home, Slate Grey (20GB RAM | 1TB SSD)
5                                   Lenovo IdeaPad 15.6" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.

In [100]:
# Vamos visualizar um registro
df_dados_notebooks['Nome'][3]

'ASUS VivoBook 15 F515 Thin and Light Laptop, 15.6” FHD Display, Intel Core i3-1005G1 Processor, Fingerprint Reader, Backlit Keyboard, Windows 10 Home, Slate Grey (20GB RAM | 1TB SSD)'

Agora sim! Primeiro, extrairemos os detalhes do processador e criaremos uma nova coluna com essas informações. Se não compreender a regra usada abaixo com expressão regular, então visite a documentação oficial:

https://docs.python.org/3/library/re.html

In [101]:
# Regra para buscar o padrão na string do Processador i3,i5,i7,i9,...
regra_busca_processador = r'\b([iI][\d])\b'

In [102]:
# Cria a coluna para o processador
df_dados_notebooks['Processador'] = df_dados_notebooks['Nome'].str.extract(regra_busca_processador)

Vamos padronizar o nome do processador substituindo I5 por i5, por exemplo.

In [103]:
df_dados_notebooks['Processador'] = df_dados_notebooks['Processador'].str.replace('I', 'i')

In [104]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco_Final,Avaliacao_Final,Processador
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,NaN
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,NaN
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,NaN
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 15.6” FHD Display, Intel Core i3-1005G1 Processor, Fingerprint Reader, Backlit Keyboard, Windows 10 Home, Slate Grey (20GB RAM | 1TB SSD)",500.0,4.3,i3
5,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,NaN


Valores nulos foram gerados, provavelmente porque alguns laptops devem ter outro tipo de processador.
Usaremos o mesmo processo para extrair informações sobre a memória RAM.

## Vamos extrair no nome do Notebook outros tipos de processadores para nossas análise
### Esta varredura buscará o texto **dual core** dentro do nome e colocará como conteúdo na Coluna Processador

In [105]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('dual core' in str_lower_nome) | ('dual-core' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'Dual Core'

df_dados_notebooks['Processador'].value_counts(dropna = False)

NaN          99
i3           39
Dual Core    38
i7           30
i5           23
i9            2
Name: Processador, dtype: int64

### Esta varredura buscará o texto **quad core** dentro do nome e colocará como conteúdo na Coluna Processador

In [106]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('quad core' in str_lower_nome) | ('quad-core' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'Quad Core'

df_dados_notebooks['Processador'].value_counts(dropna = False)

NaN          79
i3           39
Dual Core    38
i7           30
i5           23
Quad Core    20
i9            2
Name: Processador, dtype: int64

### Esta varredura buscará o texto **octa core** dentro do nome e colocará como conteúdo na Coluna Processador

In [107]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('octa core' in str_lower_nome) | ('octa-core' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'Octa Core'
        
df_dados_notebooks['Processador'].value_counts(dropna = False)

NaN          78
i3           39
Dual Core    38
i7           30
i5           23
Quad Core    20
i9            2
Octa Core     1
Name: Processador, dtype: int64

### Esta varredura buscará o texto **amd ryzen** dentro do nome e colocará como conteúdo na Coluna Processador

In [108]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('amd ryzen' in str_lower_nome) | ('amd-ryzen' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'AMD Ryzen'

df_dados_notebooks['Processador'].value_counts(dropna = False)

NaN          58
i3           39
Dual Core    38
i7           30
i5           23
AMD Ryzen    20
Quad Core    20
i9            2
Octa Core     1
Name: Processador, dtype: int64

### Esta varredura buscará o texto **amd athlon** dentro do nome e colocará como conteúdo na Coluna Processador

In [109]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('amd athlon' in str_lower_nome) | ('amd-athlon' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'AMD Athlon'

df_dados_notebooks['Processador'].value_counts(dropna = False)

NaN           55
i3            39
Dual Core     38
i7            30
i5            23
AMD Ryzen     20
Quad Core     20
AMD Athlon     3
i9             2
Octa Core      1
Name: Processador, dtype: int64

### Esta varredura buscará o texto ***pentium*** dentro do nome e colocará como conteúdo na Coluna Processador

In [110]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('pentium' in str_lower_nome) | ('intel pentium' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'Pentium'

df_dados_notebooks['Processador'].value_counts(dropna = False)

NaN           44
i3            39
Dual Core     38
i7            30
i5            23
AMD Ryzen     20
Quad Core     20
Pentium       11
AMD Athlon     3
i9             2
Octa Core      1
Name: Processador, dtype: int64

### Esta varredura buscará o texto ***celeron*** dentro do nome e colocará como conteúdo na Coluna Processador

In [111]:
for row in df_dados_notebooks.loc[pd.isna(df_dados_notebooks.Processador)].itertuples():
    str_lower_nome = str(row.Nome).lower()

    if (('celeron' in str_lower_nome) | ('intel celeron ' in str_lower_nome)):
        
        df_dados_notebooks.at[row.Index, 'Processador'] = 'Celeron'

df_dados_notebooks['Processador'].value_counts(dropna = False)

i3            39
Dual Core     38
Celeron       37
i7            30
i5            23
AMD Ryzen     20
Quad Core     20
Pentium       11
NaN            7
AMD Athlon     3
i9             2
Octa Core      1
Name: Processador, dtype: int64

In [112]:
# Regra de extração   
regra_busca_ram = r'\b([\d]+)[GB]+[ ][\+ LlPpDdRrAaMmEeOoYy\d]+\b'

In [113]:
# Extrai os dados e cria nova coluna
df_dados_notebooks['Memoria_RAM'] = df_dados_notebooks['Nome'].str.extract(regra_busca_ram)

In [114]:
# Função para limpar a coluna Memoria_RAM
def limpa_ram(ram):
    ram = pd.to_numeric(ram)
    return ram

In [115]:
# Aplica a função de limpeza
df_dados_notebooks['Memoria_RAM'] = df_dados_notebooks['Memoria_RAM'].apply(limpa_ram)

In [116]:
# Visualiza os dados
df_dados_notebooks.head()

,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 15.6” FHD Display, Intel Core i3-1005G1 Processor, Fingerprint Reader, Backlit Keyboard, Windows 10 Home, Slate Grey (20GB RAM | 1TB SSD)",500.0,4.3,i3,20.0
5,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0


Por fim, vamos extrair o tamanho da tela do notebook.

In [117]:
# Regra de extração
regra_busca_tela = r'\b([1][\d]+[\.IiNnCcHh\d]*)[ \d]*\b' 

In [118]:
df_dados_notebooks['Tamanho_Tela'] = df_dados_notebooks['Nome'].str.extract(regra_busca_tela)

In [119]:
# Buscando o total de registros por Tamanho_Tela
df_dados_notebooks['Tamanho_Tela'].value_counts(dropna = False)

15.6      89
14        53
15        29
11.6      13
17.3       8
17         8
14inch     8
12.4       3
16         3
11         3
14.1       3
12.4in     2
10         2
14.0       2
11800H     1
15inch     1
12         1
13.3       1
12700H     1
Name: Tamanho_Tela, dtype: int64

In [122]:
df_dados_notebooks_2 = df_dados_notebooks

In [120]:
# Função de limpeza
def limpa_tela(size):
        try:
            size = size.replace('Inch', '')
            size = size.replace('inch', '')
            size = size.replace('in', '')
            size = pd.to_numeric(size)
            return size
        except:
            return np.nan

In [123]:
# Aplica a função
df_dados_notebooks_2['Tamanho_Tela'] = df_dados_notebooks['Tamanho_Tela'].apply(limpa_tela)

In [124]:
df_dados_notebooks_2['Tamanho_Tela'].value_counts(dropna = False)

15.6    89
14.0    63
15.0    30
11.6    13
17.3     8
17.0     8
12.4     5
16.0     3
11.0     3
14.1     3
10.0     2
NaN      2
12.0     1
13.3     1
Name: Tamanho_Tela, dtype: int64

In [125]:
# Shape
df_dados_notebooks_2.shape

(231, 6)

In [126]:
# Visualiza os dados
df_dados_notebooks_2.head()

,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM,Tamanho_Tela
0,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0,15.6
1,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0,15.6
2,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0,15.6
3,"ASUS VivoBook 15 F515 Thin and Light Laptop, 15.6” FHD Display, Intel Core i3-1005G1 Processor, Fingerprint Reader, Backlit Keyboard, Windows 10 Home, Slate Grey (20GB RAM | 1TB SSD)",500.0,4.3,i3,20.0,15.0
5,"Lenovo IdeaPad 15.6"" Laptop Newest, 15.6 Inch HD Anti-Glare Display, AMD Dual-core Processor, 20GB RAM 1TB SSD, WiFi6 Bluetooth5, 9.5Hr Battery, Windows 11 +GM Accessories",400.0,4.3,Dual Core,20.0,15.6


In [127]:
df_dados_notebooks_2.to_csv('df_dados_notebooks_2.csv', encoding = "utf-8", index = False)

In [128]:
df = pd.read_csv("df_dados_notebooks_2.csv", encoding = "utf-8")

In [129]:
df.shape

(231, 6)

## Analytics

Estamos prontos para realizar nossa análise!

## Algumas Análises...

## Lembre-se, toda a análise refletirá o momento da busca! Pouco tempo depois, com um nova consulta, tudo pode modificar!

### 1. Quais os notes mais bem avaliados - TOP 5 - Excluíndo Celeron e Pentium?
### 2. Qual a média de valor dos notes i7?
### 3. Qual o menor valor dos notes i9?
### 4. Quais note Quad Core estão com avaliação acima de 4.5?

## ***Podem ser infinitas análises!!!***

### 1. Quais os notes mais bem avaliados - TOP 10 - Excluíndo Celeron e Pentium?

In [141]:
df[(df['Processador']!='Celeron') & (df['Processador'] != 'Pentium')].sort_values(by='Avaliacao_Final', ascending=False)[:10]


,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM,Tamanho_Tela
214,"ASUS Vivobook S 14 Flip 14” WUXGA 16:10 Touch, AMD Ryzen 5 5600H CPU, 16GB RAM, 512GB SSD, NumberPad, Fingerprint Sensor, Stylus Supported, Windows 11 Home, Quiet Blue, TN3402QA-DB56T",881.0,5.0,AMD Ryzen,16.0,14.0
119,"Lenovo Ideapad 15.6"" HD Laptop, Athlon Silver 3050U (Beats i3-1005G1) Dual-core Processor, 8GB RAM, 128GB SSD, WiFi, Webcam, Bluetooth, Win 11 S, Cloud Grey, TGC Bundle",290.0,5.0,i3,8.0,15.6
171,"OEM Lenovo ThinkPad E15 Gen 2 15.6"" FHD IPS, AMD Ryzen 7 4700U Octa Core (Beats Intel i7-1255U), 40GB RAM, 1TB NVMe, W10P, Business Laptop",900.0,5.0,i7,40.0,15.6
200,"ASUS Vivobook S 14 Flip 14” WUXGA 16:10 Touch, AMD Ryzen 5 5600H CPU, 16GB RAM, 512GB SSD, NumberPad, Fingerprint Sensor, Stylus Supported, Windows 11 Home, Quiet Blue, TN3402QA-DB56T",881.0,5.0,AMD Ryzen,16.0,14.0
216,"ASUS VivoBook Pro 15X Laptop, 144Hz 15.6” FHD Display, AMD Ryzen 9 6900HX Mobile CPU, NVIDIA GeForce RTX 3070 GPU, 32GB DDR5 RAM, 1TB PCIe SSD, Windows 11 Home, 0°Black, M6501RR-DB96",1500.0,4.8,AMD Ryzen,32.0,15.6
49,"Lenovo ThinkBook 15 Premium Business Laptop, AMD Ryzen 5 5500U Processor (6 Cores, 4.0GHz), 15.6"" FHD IPS Anti-Glare Display, 20GB RAM, 1TB SSD, Backlit Keyboard, Fingerprint Reader, Windows 11 Pro",649.0,4.7,AMD Ryzen,20.0,15.0
131,"Lenovo Flex 3 Chromebook 2-in-1 11.6"" HD (1366 x 768) Touchscreen Laptop (4-Core MediaTek MT8173C, 4GB RAM, 128GB (64GB eMMC+64GB SD Card)) flip-and-fold, Type-C, Webcam, Chrome OS",220.0,4.7,NaN,4.0,11.6
217,"Dell Newest Inspiron 15 3511 Laptop, 15.6"" FHD Touchscreen, Intel Core i5-1035G1, 32GB RAM, 1TB PCIe NVMe M.2 SSD, SD Card Reader, Webcam, HDMI, WiFi, Windows 11 Home, Black",589.0,4.7,i5,32.0,15.0
36,"HP Newest 15.6"" Laptop , FHD, AMD R5-5500U, 8GB RAM, 256GB SSD, USB Type-C and USB Type-A , HDMI, Built-in Microphone/ Webcam, Windows 11, Spruce Blue",459.0,4.7,NaN,8.0,15.6
9,"Lenovo ThinkBook 15 Premium Business Laptop, AMD Ryzen 5 5500U Processor (6 Cores, 4.0GHz), 15.6"" FHD IPS Anti-Glare Display, 20GB RAM, 1TB SSD, Backlit Keyboard, Fingerprint Reader, Windows 11 Pro",649.0,4.7,AMD Ryzen,20.0,15.0


### 2. Qual a média de valor dos notes i7?

In [143]:
df[(df['Processador']=='i7')]['Preco_Final'].mean()

868.4333333333333

In [144]:
df[(df['Processador']=='i7')]['Preco_Final'].describe()

count      30.000000
mean      868.433333
std       261.758731
min       549.000000
25%       599.000000
50%       829.000000
75%      1081.750000
max      1300.000000
Name: Preco_Final, dtype: float64

### 3. Qual o menor valor dos notes i9?

In [145]:
df[(df['Processador']=='i9')]['Preco_Final'].min()

2045.0

### 4. Quais note Quad Core estão com avaliação igual ou maior que 4.5?

In [150]:
df[(df['Processador']=='Quad Core') & (df['Avaliacao_Final'] >= 4.5)].sort_values(by='Avaliacao_Final', ascending=False)

,Nome,Preco_Final,Avaliacao_Final,Processador,Memoria_RAM,Tamanho_Tela
89,"jumper Laptop 14 Inch, 12GB DDR4 256GB SSD, Thin Light Computer with FHD 1080p Display, Intel Quad Core Processor, Windows 11 Laptops, Dual-Band WiFi(2.4G/5G), Dual Speakers, Webcam, 35520mWH Battery",290.0,4.7,Quad Core,12.0,14.0
112,"Chicbuy Laptop 15.6''12GB DDR4 512GB SSD Quad-Core Intel Celeron N5095 Processors Windows 11 Laptop Computer,1080P IPS Full HD Laptop,USB 3.0,Up to 2.9Ghz,Bluetooth 4.2,2.4G/5G WiFi,Long Battery Life",296.0,4.6,Quad Core,12.0,15.6
128,"Tulasi 12GB RAM 256GB SSD Intel Celeron N4120 Quad-Core Windows 11 Laptop Computers, 14 inch Full HD 1080P IPS Display Ultra Slim Laptop, 2.4G/5G WiFi, Bluetooth 4.2, Webcam, Long Battery Life",299.0,4.6,Quad Core,12.0,11.0
41,"Lenovo 2023 Newest Ideapad 1i Laptop, 14 inch HD Display, Intel Quad-Core Processor, 4GB RAM, 256GB Storage, Wi-Fi 6, Bluetooth, Cloud Grey, Windows 11 Home in S Mode, Bundle with JAWFOAL",220.0,4.5,Quad Core,4.0,14.0
81,"Lenovo 2023 Newest Ideapad 1i Laptop, 14 inch HD Display, Intel Quad-Core Processor, 4GB RAM, 256GB Storage, Wi-Fi 6, Bluetooth, Cloud Grey, Windows 11 Home in S Mode, Bundle with JAWFOAL",220.0,4.5,Quad Core,4.0,14.0


In [152]:
df[(df['Processador']=='Quad Core') & (df['Avaliacao_Final'] >= 4.5)].sort_values(by='Avaliacao_Final', ascending=False)['Preco_Final'].describe()

count      5.000000
mean     265.000000
std       41.206796
min      220.000000
25%      220.000000
50%      290.000000
75%      296.000000
max      299.000000
Name: Preco_Final, dtype: float64